In [2]:
import ee

In [3]:
ee.Authenticate()


True

In [4]:
ee.Initialize()

In [ ]:
aoi=ee.Geometry.Polygon(
[[39.26196452651823,21.893376873910263],
[39.26283624444807,21.893376873910263],
[39.26283624444807,21.893989109128505],
[39.26196452651823,21.893989109128505],
[39.26196452651823,21.893376873910263]]
)#UJ

In [9]:
aoi=ee.Geometry.Polygon([
[39.23103011417153,21.58115245787403],
[39.23211104440453,21.58115245787403],
[39.23211104440453,21.581850826461693],
[39.23103011417153,21.581850826461693],
[39.23103011417153,21.58115245787403]])
 #alsamer

In [11]:
aoi=ee.Geometry.Polygon([
[39.23207617568734,21.580518934885518],
[39.23315710592034,21.580518934885518],
[39.23315710592034,21.581217306527567],
[39.23207617568734,21.581217306527567],
[39.23207617568734,21.580518934885518]])
 #alsamer2

In [4]:
aoi=ee.Geometry.Polygon([

[-56.938467589094415,-8.57659829404617],
[-56.93717208214022,-8.57659829404617],
[-56.93717208214022,-8.576014806325485],
[-56.938467589094415,-8.576014806325485],
[-56.938467589094415,-8.57659829404617]

])
#amazon

In [5]:
aoi=ee.Geometry.Polygon([

[14.233839242930069,47.20476788905588],
[14.460432260508194,47.20476788905588],
[14.460432260508194,47.29239615721122],
[14.233839242930069,47.29239615721122],
[14.233839242930069,47.20476788905588]])
#ausralia

In [6]:
def maskClouds(image):
    # Example of a simple cloud mask using the QA60 band
    # Actual implementations can be more complex and might use additional bands or indices
    qa60 = image.select('QA60')
    cloudMask = qa60.bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloudMask)

In [18]:
# Filter the Sentinel-2 ImageCollection for the specified date range, AOI, and apply cloud masking
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2023-01-01', '2024-01-30') \
    .filterBounds(aoi) \
    .map(maskClouds) \
    .select(['B4', 'B8']) \
    .median()  # Compute the median of the filtered collection

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast selected bands to Float32
def castImage(image):
    return image.toFloat()

# Cast the selected bands of the clipped image to Float32
castedImage = castImage(clippedImage)

# Example export parameters (make sure to replace 'aoi.getInfo()['coordinates']' with your AOI coordinates if necessary)
task = ee.batch.Export.image.toDrive(**{
    'image': castedImage,
    'description': 'australia',
    'scale': 10,
    'region': aoi.getInfo()['coordinates'],  # Make sure this is correctly defined for your AOI
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})
task.start()

# another way

In [7]:
# Filter the Sentinel-2 ImageCollection for images within the AOI and time frame, and compute the median
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate('2023-01-01', '2024-01-01') \
    .filterBounds(aoi) \
    .median()

# Clip the median image to the AOI
clippedImage = sentinelCollection.clip(aoi)

# Function to cast all bands to Float32
def castImage(image):
    return image.toFloat()

# Cast all bands of the clipped image to Float32
castedImage = castImage(clippedImage)


 #Export the casted image, specifying scale and region (this is an example, see below for details)
task = ee.batch.Export.image.toDrive(**{
   'image': castedImage,
   'description': 'australia-2',
   'scale': 10,
   'region': aoi.getInfo()['coordinates'],
   'fileFormat': 'GeoTIFF',
   'maxPixels': 1e9
 })
task.start()